In [1]:
import pandas as pd
import os
import numpy as np

# Load data after 2017

In [2]:
directory = r"C:\Users\olive\Documents\GitHub\MilaClimateChangeHackathon2019\data"
df = pd.DataFrame()
for dirpath, dirnames, filenames in os.walk(directory):
    for file in filenames:
        if '_standard.csv' in file:
            abspath = dirpath + '\\' + file
            dfT = pd.read_csv(abspath, index_col='time', parse_dates=True)
            dfT = dfT.loc['2018':]
            df = pd.concat([df, dfT])

C:\Users\olive\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


# One-Hot Encode Grids

In [3]:
# Add column Grid to keep track of what grid the data is from
df['Grid'] = df['lat'].astype('str') + '|' + df['lon'].astype('str')
# One-Hot Encode Grid to use as feature later
df = pd.concat([df, pd.get_dummies(df['Grid'], prefix='GRID', drop_first=True)], axis=1)

# One-Hot Encode Time Information

In [4]:
df['Weekday'] = df.index.weekday
df = pd.concat([df.drop(['Weekday'], axis=1), pd.get_dummies(df['Weekday'], prefix='WEEKDAY', drop_first=True)], axis=1)

df['Month'] = df.index.month
df = pd.concat([df.drop(['Month'], axis=1), pd.get_dummies(df['Month'], prefix='MONTH', drop_first=True)], axis=1)

# Create Function to Return Closest Grid for a Coordinate

In [5]:
def closest_grid(lat, lon, grids):
    point = np.array([float(lat), float(lon)])
    min_dist = 1_000_000_000_000
    for grid in grids:
        lat, lon = grid.replace('GRID_', '').split('|')
        grid_array = np.array([float(lat), float(lon)])
        dist = np.sum((point - grid_array)**2)
        if dist < min_dist:
            min_dist = dist
            best_grid = grid
    return best_grid

# Load Precipitation Data & extract used grids

In [6]:
pre_df = pd.read_csv(r"C:\Users\olive\Documents\GitHub\MilaClimateChangeHackathon2019\data\prepared_precipitation_data.csv",
                    index_col=0)

In [7]:
grids = [x for x in pre_df.columns if 'GRID_' in x]

# Snap All Coordinates to a Grid

In [8]:
df['Grid'] = df.apply(lambda row: closest_grid(row['lat'], row['lon'], grids), axis=1)

# Send this dataset to CSV in the same format as predictor arrays X to be used to forecast later

In [9]:
feature_cols = ['pr_CNRM-CM5_historical_rcp26_(kg m-2 d-1)',
       'pr_CNRM-CM5_historical_rcp45_(kg m-2 d-1)',
       'pr_CNRM-CM5_historical_rcp85_(kg m-2 d-1)',
       'pr_MIROC-ESM-CHEM_historical_rcp26_(kg m-2 d-1)',
       'pr_MIROC-ESM-CHEM_historical_rcp45_(kg m-2 d-1)',
       'pr_MIROC-ESM-CHEM_historical_rcp85_(kg m-2 d-1)',
       'pr_MIROC-ESM_historical_rcp26_(kg m-2 d-1)',
       'pr_MIROC-ESM_historical_rcp45_(kg m-2 d-1)',
       'pr_MIROC-ESM_historical_rcp85_(kg m-2 d-1)',
       'pr_MRI-CGCM3_historical_rcp26_(kg m-2 d-1)',
       'pr_MRI-CGCM3_historical_rcp45_(kg m-2 d-1)',
       'pr_MRI-CGCM3_historical_rcp85_(kg m-2 d-1)',
       'GRID_45.458333155|-73.625002695',
       'GRID_45.458333155|-73.70833602500002',
       'GRID_45.458333155|-73.791669355', 'GRID_45.458333155|-73.875002685',
       'GRID_45.458333155|-73.958336015', 'GRID_45.541666485|-73.541669365',
       'GRID_45.541666485|-73.625002695',
       'GRID_45.541666485|-73.70833602500002',
       'GRID_45.541666485|-73.791669355', 'GRID_45.541666485|-73.875002685',
       'GRID_45.624999815|-73.541669365', 'GRID_45.624999815|-73.625002695',
       'GRID_45.624999815|-73.70833602500002',
       'GRID_45.624999815|-73.791669355', 'WEEKDAY_1', 'WEEKDAY_2',
       'WEEKDAY_3', 'WEEKDAY_4', 'WEEKDAY_5', 'WEEKDAY_6', 'MONTH_2',
       'MONTH_3', 'MONTH_4', 'MONTH_5', 'MONTH_6', 'MONTH_7', 'MONTH_8',
       'MONTH_9', 'MONTH_10', 'MONTH_11', 'MONTH_12']

In [10]:
future_data = df[feature_cols]

In [11]:
future_data.head()

,pr_CNRM-CM5_historical_rcp26_(kg m-2 d-1),pr_CNRM-CM5_historical_rcp45_(kg m-2 d-1),pr_CNRM-CM5_historical_rcp85_(kg m-2 d-1),pr_MIROC-ESM-CHEM_historical_rcp26_(kg m-2 d-1),pr_MIROC-ESM-CHEM_historical_rcp45_(kg m-2 d-1),pr_MIROC-ESM-CHEM_historical_rcp85_(kg m-2 d-1),pr_MIROC-ESM_historical_rcp26_(kg m-2 d-1),pr_MIROC-ESM_historical_rcp45_(kg m-2 d-1),pr_MIROC-ESM_historical_rcp85_(kg m-2 d-1),pr_MRI-CGCM3_historical_rcp26_(kg m-2 d-1),...,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12
time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 12:00:00,-0.000092,4.833954,0.878815,-0.000092,7.116730,-0.000092,-0.000092,2.477966,-0.000092,1.965271,...,0,0,0,0,0,0,0,0,0,0
2018-01-02 12:00:00,-0.000092,5.676270,9.863342,-0.000092,8.264191,2.685486,-0.000092,-0.000092,3.332489,10.473694,...,0,0,0,0,0,0,0,0,0,0
2018-01-03 12:00:00,-0.000092,-0.000092,-0.000092,-0.000092,3.515595,5.603027,-0.000092,3.173798,3.967255,1.037537,...,0,0,0,0,0,0,0,0,0,0
2018-01-04 12:00:00,0.524811,14.306763,-0.000092,-0.000092,0.183014,-0.000092,-0.000092,-0.000092,1.745544,0.378326,...,0,0,0,0,0,0,0,0,0,0
2018-01-05 12:00:00,-0.000092,-0.000092,9.912170,-0.000092,0.414948,-0.000092,-0.000092,-0.000092,-0.000092,-0.000092,...,0,0,0,0,0,0,0,0,0,0


In [13]:
future_data.to_csv(r"C:\Users\olive\Documents\GitHub\MilaClimateChangeHackathon2019\data\Future_Dataset_WithDates.csv")